In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


!apt-get update -qq && apt-get install -qq poppler-utils
!pip install easyocr pdf2image opencv-python pillow --quiet


!mkdir -p /root/.EasyOCR/model
!wget -q -O /root/.EasyOCR/model/craft_mlt_25k.pth \
    "https://huggingface.co/xiaoyao9184/easyocr/resolve/master/craft_mlt_25k.pth"
!wget -q -O /root/.EasyOCR/model/bengali.pth \
    "https://huggingface.co/xiaoyao9184/easyocr/resolve/master/bengali.pth"


Mounted at /content/drive
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:

import easyocr
reader = easyocr.Reader(
    ['bn'],
    gpu=False,
    download_enabled=False,
    model_storage_directory='/root/.EasyOCR/model'
)


In [ ]:

from pdf2image import convert_from_path
import cv2, numpy as np, os

PDF_PATH = "/content/drive/MyDrive/MyDrive/Book.pdf"
OUT_DIR = '/content/drive/MyDrive/ocr_output'
os.makedirs(OUT_DIR, exist_ok=True)

pages = convert_from_path(PDF_PATH, dpi=300)

for i, page in enumerate(pages):
    # Preprocess page image
    img = cv2.cvtColor(np.array(page), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bw = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 21, 8)

    # OCR extraction
    text_lines = reader.readtext(bw, detail=0, paragraph=True)
    text = "\n".join(text_lines)

    # Save output
    fn = f'{OUT_DIR}/page_{i+1:03d}.txt'
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(text)
    print(f'✅ Saved: {fn}')


✅ Saved: /content/drive/MyDrive/ocr_output/page_001.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_002.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_003.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_004.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_005.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_006.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_007.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_008.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_009.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_010.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_011.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_012.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_013.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_014.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_015.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_016.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_017.txt
✅ Saved: /content/drive/MyDrive/ocr_output/page_

In [ ]:
!pip install pymupdf


In [ ]:
import fitz  # PyMuPDF
import os

pdf_path = "/content/drive/MyDrive/MyDrive/Book.pdf"
output_dir = "/content/drive/MyDrive/normal_pages"
os.makedirs(output_dir, exist_ok=True)

with fitz.open(pdf_path) as doc:
    for i, page in enumerate(doc):
        page_text = page.get_text()
        with open(f"{output_dir}/page_{i+1:03d}.txt", "w", encoding="utf-8") as f:
            f.write(page_text)



In [ ]:
!pip install openai --quiet
import openai
import os


In [ ]:
import openai

ocr_dir = "/content/drive/MyDrive/ocr_output"
normal_dir = "/content/drive/MyDrive/normal_pages"
cleaned_dir = "/content/drive/MyDrive/cleaned_pages"
os.makedirs(cleaned_dir, exist_ok=True)

import os
OPENAI_API_KEY= ""

for i in range(1, 50):
    ocr_file = f"{ocr_dir}/page_{i:03d}.txt"
    normal_file = f"{normal_dir}/page_{i:03d}.txt"

    # Read both files
    with open(ocr_file, encoding="utf-8") as f:
        ocr_text = f.read()
    with open(normal_file, encoding="utf-8") as f:
        normal_text = f.read()

    # Compose LLM prompt
    prompt = f"""
You are a highly skilled Bangla document reconstruction and cleaning assistant.
You are provided with two versions of the same Bangla textbook content:
- OCR-extracted text (which may contain recognition errors, missing words, or artifacts)
- Direct PDF-extracted text (which may contain gibberish or broken Unicode)

Task:
- Carefully compare both versions and reconstruct the most accurate, readable, and well-formatted Bangla text.
- If you detect that any part of the content is or should be a table, MCQ (multiple choice question), or options for a question, format them properly (use markdown tables, or clear bullet/numbered lists for options and MCQs).
- For MCQs: ensure that each question and its options are clearly separated and labeled.
- For tables: reconstruct using markdown (| column | column | ...) if possible.
- Where both files are noisy, do your best to guess the most plausible reconstruction based on context.
- Remove repeated headers, page numbers, and irrelevant artifacts.
- Make the paragraphs and lists flow naturally, ready for use in a Bangla knowledge retrieval system.

Here are the two versions:
[OCR_OUTPUT]
{ocr_text}

[NORMAL_EXTRACTION]
{normal_text}

Please return the best possible, clean, organized Bangla text as described.
"""

    client = openai.OpenAI(api_key=OPENAI_API_KEY)

    response = client.chat.completions.create(
      model="gpt-4.1-mini",
      messages=[
        {"role": "system", "content": "You are an expert at reconstructing and formatting Bangla educational documents."},
        {"role": "user", "content": prompt}
      ],
      temperature=0.2,
      max_tokens=2064
)
    cleaned_text = response.choices[0].message.content


    # Save cleaned page
    with open(f"{cleaned_dir}/page_{i:03d}.txt", "w", encoding="utf-8") as f:
        f.write(cleaned_text)
    print(f"✅ Cleaned page saved: page_{i:03d}.txt")


✅ Cleaned page saved: page_001.txt
✅ Cleaned page saved: page_002.txt
✅ Cleaned page saved: page_003.txt
✅ Cleaned page saved: page_004.txt
✅ Cleaned page saved: page_005.txt
✅ Cleaned page saved: page_006.txt
✅ Cleaned page saved: page_007.txt
✅ Cleaned page saved: page_008.txt
✅ Cleaned page saved: page_009.txt
✅ Cleaned page saved: page_010.txt
✅ Cleaned page saved: page_011.txt
✅ Cleaned page saved: page_012.txt
✅ Cleaned page saved: page_013.txt
✅ Cleaned page saved: page_014.txt
✅ Cleaned page saved: page_015.txt
✅ Cleaned page saved: page_016.txt
✅ Cleaned page saved: page_017.txt
✅ Cleaned page saved: page_018.txt
✅ Cleaned page saved: page_019.txt
✅ Cleaned page saved: page_020.txt
✅ Cleaned page saved: page_021.txt
✅ Cleaned page saved: page_022.txt
✅ Cleaned page saved: page_023.txt
✅ Cleaned page saved: page_024.txt
✅ Cleaned page saved: page_025.txt
✅ Cleaned page saved: page_026.txt
✅ Cleaned page saved: page_027.txt
✅ Cleaned page saved: page_028.txt
✅ Cleaned page saved

In [ ]:
!pip install sentence-transformers



In [ ]:
!pip install -U pinecone sentence-transformers

In [ ]:
#PINECONE_API_KEY = "pcsk_2czr3A_58ustdWMomnjZ7p1P3P5UaNPagR9h61WNiW6iAeXFiDkH3gWrNoqpbxFghTZYCY"

In [ ]:

import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

# Pinecone credentials
PINECONE_API_KEY = "pcsk_2czr3A_58ustdWMomnjZ7p1P3P5UaNPagR9h61WNiW6iAeXFiDkH3gWrNoqpbxFghTZYCY"
INDEX_NAME = "bangla-book-index"
DIMENSION = 1024


pc = Pinecone(api_key=PINECONE_API_KEY)


if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=DIMENSION,
        metric='cosine',
        spec=ServerlessSpec(
             cloud="aws",
             region="us-east-1"
        )
    )

#  Connect to index
index = pc.Index(INDEX_NAME)

# Load E5-large model
model = SentenceTransformer('intfloat/multilingual-e5-large')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
CLEANED_DIR = "/content/drive/MyDrive/cleaned_pages"
chunks = []
chunk_ids = []

for fname in sorted(os.listdir(CLEANED_DIR)):
    if fname.endswith(".txt"):
        with open(os.path.join(CLEANED_DIR, fname), encoding="utf-8") as f:
            text = f.read().strip()
            if text:
                chunks.append(text)
                chunk_ids.append(fname.replace('.txt', ''))  # e.g., 'page_001'

embeddings = model.encode(
    [f"passage: {chunk}" for chunk in chunks],
    show_progress_bar=True,
    convert_to_numpy=True
)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
batch_size = 32
for i in range(0, len(chunks), batch_size):
    batch_ids = chunk_ids[i:i+batch_size]
    batch_embs = embeddings[i:i+batch_size]
    batch_texts = chunks[i:i+batch_size]
    pinecone_vectors = [
        (batch_ids[j], batch_embs[j].tolist(), {"text": batch_texts[j]})
        for j in range(len(batch_ids))
    ]
    index.upsert(vectors=pinecone_vectors)
    print(f"✅ Upserted batch {i // batch_size + 1}")

print("🎉 All cleaned chunks embedded and uploaded to Pinecone!")


✅ Upserted batch 1
✅ Upserted batch 2
🎉 All cleaned chunks embedded and uploaded to Pinecone!
